In [1]:
# This script returns mass and panel sizing for a solar panel array. It was originally made for
# the 2021 UIUC RASC-AL team by Komol Patel and written in MATLAB. It was translated into Python and
# modified to support a varying power duty-cycle and account for different solar radiation around 
# the solar system for the 2022 UIUC RASC-AL team by Galen Sieck. 

#The first three blocks of code contain the values to be modified. It is currently defaulted 
#to the 2022 MITHRIL architecture which refined fuel on Mars.

#Chosen duty cycle input
totalPowerPeak =  700000 #W, this is the peak power generation required of the design
totalPowerBase = 0 #W, this is the base power generation requirement
peakPeriod = 24 #h, length of time the architecture runs at peak power level
basePeriod = 24-peakPeriod #h, length of time the architecture runs at base power
chargePeriod = 4 #h, the period of time the solar panels will generate power

#Chosen architecture parametrs
DoD = .6 #The depth of discharge for the battery packs
specificPower = 200 # W/kg, this is a feature of the exact solar cell you plan to use
massDensity = 3.8 # kg/m^2, this is the density of the solar cells on the panel surface
massAerialDensity = 5 #kg/m^2, this is the density of the solar cells themselves and how much power they can generate per unit 
solarFluxRatio = 2.3 #the ratio of solar flux at architecture location to solar flux on Earth
useTruss = False

# Truss Parameters
# this entire section is only if you want a trussed support to hold your
# solar panels up, tower style. Refer to RASC-AL 2021 design for example
# application of this section of code.
totalHeight = 13 # m, this is how high off the ground you want your panels to be
numberOfSections = 4 # this is however many sections you want your total panel area split up into
density = 2700 # kg/m^3, this is the desity of the material you intend to make your truss out of
initialCSwidth = 0.1 # m, CS = cross-section, this is the initial "guess" the code uses to iterate through the section below
CSwidthIncrement = 0.01 # m, this is the amount by which the cross-sectional width increases in each iteration


#Duty cycle intermediate values
energyReq = (totalPowerPeak*peakPeriod+totalPowerBase*basePeriod)/DoD #total required energy per discharge period
totalPowerNeed = energyReq/chargePeriod #total power required for the duration of the charge period

#Panel Mass
panelArea = totalPowerNeed / (specificPower/solarFluxRatio * massAerialDensity) # this is the area your solar panel needs to be to generate the required amount of power
panelMass = massDensity*panelArea # given the required solar panel area calculated above, this is the mass of your panels

#Battery Mass
specificEnergy = 200
batteryMass = energyReq/specificEnergy

#Truss Sizing
trussMassFraction = 0.40; # this determines how much material is actually in the truss compared to if it were a solid-walled tube.
postMass = 0; # this is just the inital "guess" for iterative sake
sectionLength = totalHeight / numberOfSections; # this finds the length of each section, assuming all sections are evenly sized
width = initialCSwidth; # this is just the iterative initial "guess"
for i in range(1,numberOfSections):
    sectionMass = (width**2 - (width-CSwidthIncrement)**2)*sectionLength*trussMassFraction*density; # this calculates the mass of a single truss section
    postMass = postMass + sectionMass; # this increases increases the post mass by another section mass, effectively "adding" a section
    width = width - CSwidthIncrement; # this makes the width skinnier, which is beneficial to reducing mass - we don't want this to be negative though (refer to if statement below)
    if width <= 0: # this is a check to make sure we don't end up with negative dimensions on our supports
        error('ERROR: width is no longer positive. Make initial width bigger or increment smaller.')
trussMass = sectionMass+postMass

#Outputs
if useTruss == True:
    totalMass = trussMass + panelMass + batteryMass
else:
    totalMass = panelMass + batteryMass

print(totalMass, "kg")

201180.0 kg
